# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
sys.path.append("../")


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('city_weather.csv')
df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,albany,100.0,US,1.622322e+09,72.0,42.6001,-73.9662,53.15,5.37
1,victor harbor,0.0,AU,1.622322e+09,70.0,-35.5667,138.6167,49.80,8.16
2,cape town,40.0,ZA,1.622321e+09,87.0,-33.9258,18.4232,58.08,0.07
3,takayama,70.0,JP,1.622321e+09,90.0,36.1333,137.2500,52.03,1.57
4,morant bay,11.0,JM,1.622322e+09,73.0,17.8815,-76.4093,85.62,8.84
...,...,...,...,...,...,...,...,...,...
559,granada,73.0,ES,1.622322e+09,55.0,37.2500,-3.2500,68.29,5.53
560,college,75.0,US,1.622322e+09,44.0,64.8569,-147.8028,52.59,1.01
561,cleethorpes,0.0,GB,1.622322e+09,92.0,53.5605,-0.0323,59.09,6.76
562,opuwo,1.0,NaN,1.622322e+09,28.0,-18.0607,13.8400,62.78,6.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = df[['Lat', 'Lng']].astype(float)
humidity = df['Humidity'].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = df[(df['Max Temp'] >= 70) & (df['Max Temp'] <= 80) & (df['Wind Speed'] <= 10) & (df['Cloudiness'] < 10)]
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
113,hoquiam,1.0,US,1.622321e+09,49.0,46.9809,-123.8893,73.92,8.05
152,shieli,7.0,KZ,1.622322e+09,36.0,44.1667,66.7500,78.01,6.35
182,mitsamiouli,4.0,KM,1.622322e+09,73.0,-11.3847,43.2844,78.39,5.97
285,gat,0.0,IL,1.622322e+09,100.0,31.6100,34.7642,72.39,2.33
349,tonekabon,0.0,IR,1.622322e+09,53.0,36.8163,50.8738,73.35,3.36
372,mangaldai,9.0,IN,1.622322e+09,82.0,26.4333,92.0333,78.22,3.65
479,winnemucca,1.0,US,1.622322e+09,8.0,40.9730,-117.7357,77.05,8.61
492,namibe,0.0,AO,1.622322e+09,75.0,-15.1961,12.1522,72.25,6.40
523,aran,0.0,AZ,1.622322e+09,88.0,40.6253,46.9756,78.48,9.84


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
geocode_base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
query_params = {}
query_params['key'] = g_key
query_params['radius'] = 7000
query_params['type'] = 'lodging'
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_locations_list = []
hotel_names = []
addresses = []
for index, row in ideal_df.iterrows():
    target_coords = f"{row['Lat']},{row['Lng']}"
    query_params['location'] = target_coords
    response = requests.get(base_url, params=query_params).json()
    try:
        lat = float(response['results'][0]['geometry']['location']['lat'])
        lon = float(response['results'][0]['geometry']['location']['lng'])
        name = response['results'][0]['name']
        hotel_names.append(name)
        hotel_locations_list.append((lat,lon))
        params = {}
        params['key'] = g_key
        params['latlng'] = f"{lat},{lon}"
        params['rankby'] = 'distance'
        response = requests.get(geocode_base_url, params=params).json()
        address = response['results'][0]['formatted_address']
        addresses.append(address)
    except Exception as e:
        print(e)
hotel_df = pd.DataFrame({'Hotel Name': hotel_names, 'Address': addresses})
hotel_df

list index out of range


,Hotel Name,Address
0,Olympic Inn & Suites,"616 W Heron St, Aberdeen, WA 98520, USA"
1,Gostinitsa Aybek Shieli,"Chiili, Shieli, Kazakhstan"
2,Foyer ADM,"Mitsamiouli Djao، Mitsamiouli, Comoros"
3,OlusHome,"68, Nir Hen, Israel"
4,Hotel Sadra,"Mazandaran Province, Tonekabon, کریم اباد، Iran"
5,Hotel Siddarth Palace,"Mangaldoi, Santipara w/n 1, Mangaldoi, Assam 7..."
6,Winners Inn Casino,"185 W Winnemucca Blvd, Winnemucca, NV 89445, USA"
7,Hotel Chik Chik Namibe,"Unnamed Road, Namibe, Angola"


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(hotel_locations_list, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))